In [8]:
import numpy as np
import cv2 as cv
import imutils
from scipy.spatial import distance as dist
from config import *

In [9]:
labelsPath = MODEL_PATH + '/coco.names'
LABELS = open(labelsPath).read().strip().split("\n")

weightsPath = MODEL_PATH + "/yolov3.weights"
configPath = MODEL_PATH + "/yolov3.cfg"

yolo = cv.dnn.readNetFromDarknet(configPath, weightsPath)

if USE_GPU:
	print("[INFO] setting preferable backend and target to CUDA...")
	yolo.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
	yolo.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)

output_layer = yolo.getUnconnectedOutLayersNames()

In [10]:
vs = cv.VideoCapture(VIDEO_PATH)
output_video = None

In [11]:
def detect_people(frame, net, ln, personIdx):
    (H,W) = frame.shape[:2]
    results = []

    blob = cv.dnn.blobFromImage(frame, 1/255.0, (416,416), swapRB = True, crop = False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)
    boxes = []
    centroids = []
    confidences = []
    
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if classID == personIdx and confidence > MIN_CONF:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))

    idxs = cv.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

    if len(idxs) > 0:
        for i in idxs.flatten():
            (x,y) = (boxes[i][0], boxes[i][1])
            (w,h) = (boxes[i][2], boxes[i][3])

            instance = (confidences[i], (x,y,x+w, y+h), centroids[i])
            results.append(instance)
            
    return results

In [12]:
while True:
  (frame_exists, frame) = vs.read()
  if not frame_exists:
    break
  else:
    frame = imutils.resize(frame, width = SIZE_FRAME)
    results = detect_people(frame, yolo, output_layer, personIdx= LABELS.index("person"))
        
    violate = set()

    if len(results) > 2:
      centroids = np.array([r[2] for r in results])
      distance = dist.cdist(centroids, centroids, metric="euclidean")
            
      for i in range(0, distance.shape[0]):
        for j in range(i+1, distance.shape[1]):
          if distance[i,j] < MIN_DISTANCE:
            violate.add(i)
            violate.add(j)
        
    for (i, (confidence, bbox, centroid)) in enumerate(results):
      (startX, startY, endX, endY) = bbox
      (cX, cY) = centroid
      color = COLOR_GREEN
      
      if i in violate:
        color = COLOR_RED
        
      cv.rectangle(frame, (startX, startY), (endX, endY), color, 2)
      cv.circle(frame, (cX, cY), 5, color, 1)

  cv.imshow("Social Distance Detection", frame)
  key = cv.waitKey(1) & 0xFF
  
  if output_video is None:
    fourcc1 = cv.VideoWriter_fourcc(*"MJPG")
    output_video = cv.VideoWriter("./output/result.avi", fourcc1, 25,(frame.shape[1], frame.shape[0]), True)
  else:
    print("writing to video")
    output_video.write(frame)

  if key == ord("q"):
    break

hello
hello
writing to video
hello
writing to video
hello
writing to video
hello
writing to video
hello
writing to video
hello
writing to video


: 